# [DBA Mastery](http://google.com)
**Just another witchcraft and wizardry site and DBA tips …**

# Upgrading a SQL Server docker container using Python
- 1- Checking SQL Server images in MCR **(optional)**
- 2- Pull images **(optional)**
- 3- Check images **(optional)**
- 4- Create CU13 container
- 5- Check container status
- 6- Connect to CU13 and get version
- 7- Restore WWI database from backup
- 8- Stop and upgrade CU13 to CU14
- 9- Check SQL Server errorlog
- 10- Connect to CU13 and get version

This notebook uses the Docker Python API to manage a SQL Server container created in Docker.

# 1- Checking SQL Server images in MCR (Python)

In [1]:

# Importing HTTP requests library
import requests

# Listing available images at MCR
mcr_images = requests.get('https://mcr.microsoft.com/v2/mssql/server/tags/list/')
mcr_images.json()

{'name': 'mssql/server',
 'tags': ['2017-CU1-ubuntu',
  '2017-CU10',
  '2017-CU10-ubuntu',
  '2017-CU11',
  '2017-CU11-ubuntu',
  '2017-CU12',
  '2017-CU12-ubuntu',
  '2017-CU13',
  '2017-CU13-ubuntu',
  '2017-CU14',
  '2017-CU14-ubuntu',
  '2017-CU2-ubuntu',
  '2017-CU3-ubuntu',
  '2017-CU4-ubuntu',
  '2017-CU5-ubuntu',
  '2017-CU6-ubuntu',
  '2017-CU7-ubuntu',
  '2017-CU8-ubuntu',
  '2017-CU9-ubuntu',
  '2017-GA-ubuntu',
  '2017-GDR-ubuntu',
  '2017-latest',
  '2017-latest-ubuntu',
  '2018-CTP20',
  '2018-CTP20-ubuntu',
  '2019-CTP2.0',
  '2019-CTP2.0-ubuntu',
  '2019-CTP2.1',
  '2019-CTP2.1-ubuntu',
  '2019-CTP2.2',
  '2019-CTP2.2-ubuntu',
  '2019-CTP2.3',
  '2019-CTP2.3-ubuntu',
  '2019-CTP2.4',
  '2019-CTP2.4-ubuntu',
  '2019-CTP2.5',
  '2019-CTP2.5-ubuntu',
  '2019-latest',
  'latest',
  'latest-ubuntu',
  'vNext-CTP2.0-ubuntu']}

# 2- Pull images (Python)

In [11]:
# Importing docker library
import docker
import requests
client = docker.from_env()
client.images.pull('mcr.microsoft.com/mssql/server:2017-CU13-ubuntu')
print ('SQL Server 2017 \ Ubuntu CU13 has been pulled')
client.images.pull('mcr.microsoft.com/mssql/server:2017-CU14-ubuntu')
print ('SQL Server 2017 \ Ubuntu CU14 has been pulled')

SQL Server 2017 \ Ubuntu CU13 has been pulled


SQL Server 2017 \ Ubuntu CU14 has been pulled


# 3- Check images (Python)

In [7]:
# Listing local images (Docker cache)
client.images.list()


[<Image: 'mcr.microsoft.com/mssql/server:2017-CU14-ubuntu'>,
 <Image: 'mcr.microsoft.com/mssql/server:2017-CU13-ubuntu'>,
 <Image: 'microsoft/mssql-server-linux:2017-CU12', 'mcr.microsoft.com/mssql/server:2017-CU12-ubuntu'>,
 <Image: 'mcr.microsoft.com/mssql/server:2017-CU11'>,
 <Image: 'microsoft/mssql-server-linux:2017-CU11'>]

# 4- Create CU13 container (Python)

In [5]:
# Checking if existing \ Cleaning up
try:
        container = client.containers.get('PythonSQL_CU13')
        container.stop()
        container.remove()
        print ('The existing PythonSQL_CU13 container was removed ...')
except:
        print ("The PythonSQL_CU13 container does not exists ...")

# Creating Python_CU14 container
client.containers.run(
        'mcr.microsoft.com/mssql/server:2017-CU13-ubuntu',
        detach=True,
        name="PythonSQL_CU13",
        hostname="PythonSQL_CU13",
        environment={"ACCEPT_EULA":"Y","MSSQL_SA_PASSWORD":"P1th0nR0ck$!"},
        volumes= 
        {'vlm_VersionUpgrade': {'bind': '/var/opt/mssql', 'mode': 'rw'},
        '/Users/carlos/Documents/Containers': {'bind': '/Shared', 'mode': 'rw'}},
        ports={'1433/tcp': 1401}
)
print ("The PythonSQL_CU13 container was succesfully created")

phython_cu13 was removed


The container succesfully created


# 5- Check container status (Python)

In [3]:
# Setting context to my container
container = client.containers.get('PythonSQL_CU13')

# Checking status of PythonSQL container
print ('My container: ' + container.name + ' is in ' + container.status + ' state.')

# Checking status of all containers
client.containers.list(all=True, filters={"status":"running"})

My container: python_cu13 is in running state.


# 6- Connect to CU13 and get version (SQL)

In [1]:
-- Checking SQL Server instance information
-- 14.0.3048.4 = CU13
-- 14.0.3076.1 = CU14
SELECT
    SERVERPROPERTY('ServerName') AS [Instance Name],
    SERVERPROPERTY('ProductVersion') AS [Product Version],
    RIGHT(@@version, LEN(@@version)- 3 -charindex (' ON ', @@VERSION)) [OS Version],
    SERVERPROPERTY ('Edition') AS [Edition]
FROM sys.dm_os_sys_info;

# 7- Restore WWI database from backup (SQL)

In [0]:
-- Checking existing databases
SELECT name FROM sys.databases;

-- Restore WWI backup
RESTORE DATABASE WWI FROM DISK = '/Shared/wwi.bak' WITH
MOVE 'WWI_Primary' TO '/var/opt/mssql/data/WideWorldImporters.mdf',
MOVE 'WWI_UserData' TO '/var/opt/mssql/data/WideWorldImporters_userdata.ndf',
MOVE 'WWI_Log' TO '/var/opt/mssql/data/WideWorldImporters.ldf', 
MOVE 'WWI_InMemory_Data_1' TO '/var/opt/mssql/data/WideWorldImporters_InMemory_Data_1';

-- Looking for WWI database
SELECT name FROM sys.databases;

-- Checking SQL Server version --14.0.3048.4 = CU13
SELECT
    SERVERPROPERTY('ServerName') AS [Instance Name],
    SERVERPROPERTY('ProductVersion') AS [Product Version],
    RIGHT(@@version, LEN(@@version)- 3 -charindex (' ON ', @@VERSION)) [OS Version],
    SERVERPROPERTY ('Edition') AS [Edition]
FROM sys.dm_os_sys_info;

# 8- Stop and upgrade CU13 to CU14 (Python)

In [0]:
# Setting context to my container
container = client.containers.get('PythonSQL_CU13')

# Stopping PythonSQL
container.stop()

# Creating Python_CU14 container (upgrade)
client.containers.run(
        'mcr.microsoft.com/mssql/server:2017-CU14-ubuntu',
        detach=True,
        name="PythonSQL_CU14",
        hostname="PythonSQL_CU14",
        environment={"ACCEPT_EULA":"Y","MSSQL_SA_PASSWORD":"P1th0nR0ck$!"},
        volumes= 
        {'vlm_VersionUpgrade': {'bind': '/var/opt/mssql', 'mode': 'rw'},
        '/Users/carlos/Documents/Containers': {'bind': '/Shared', 'mode': 'rw'}},
        ports={'1433/tcp': 1401}
)
print ("The PythonSQL_CU14 container was succesfully created")

# 9- Check SQL Server errorlog (Python)

In [0]:
# Setting context to my container
container = client.containers.get('PythonSQL_CU14')

# Checking the log
for line in container.logs(stream=True):
    print (line.strip())


b'2019-05-05 05:44:51.00 Server      Microsoft SQL Server 2017 (RTM-CU13) (KB4466404) - 14.0.3048.4 (X64)'
b'Nov 30 2018 12:57:58'
b'Copyright (C) 2017 Microsoft Corporation'
b'Developer Edition (64-bit) on Linux (Ubuntu 16.04.5 LTS)'
b'2019-05-05 05:44:51.01 Server      UTC adjustment: 0:00'
b'2019-05-05 05:44:51.01 Server      (c) Microsoft Corporation.'
b'2019-05-05 05:44:51.01 Server      All rights reserved.'
b'2019-05-05 05:44:51.01 Server      Server process ID is 4120.'
b"2019-05-05 05:44:51.01 Server      Logging SQL Server messages in file '/var/opt/mssql/log/errorlog'."
b'2019-05-05 05:44:51.02 Server      Registry startup parameters:'
b'-d /var/opt/mssql/data/master.mdf'
b'-l /var/opt/mssql/data/mastlog.ldf'
b'-e /var/opt/mssql/log/errorlog'
b'2019-05-05 05:44:51.03 Server      SQL Server detected 2 sockets with 1 cores per socket and 1 logical processors per socket, 2 total logical processors; using 2 logical processors based on SQL Server licensing. This is an information

# 10- Connect to CU13 and get version (Python)

In [0]:
-- Checking SQL Server instance information
-- 14.0.3048.4 = CU13
-- 14.0.3076.1 = CU14
SELECT
    SERVERPROPERTY('ServerName') AS [Instance Name],
    SERVERPROPERTY('ProductVersion') AS [Product Version],
    RIGHT(@@version, LEN(@@version)- 3 -charindex (' ON ', @@VERSION)) [OS Version],
    SERVERPROPERTY ('Edition') AS [Edition]
FROM sys.dm_os_sys_info;


# Follow me
- Twitter:&nbsp;&nbsp;&nbsp;&nbsp;[@dbamastery](https://twitter.com/dbamastery)  
- GitHub:&nbsp;&nbsp;&nbsp;&nbsp;[dbamaster](https://github.com/dbamaster)  
- Blog:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;http://dbamastery.com
